In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("data.csv")
print(df)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print(df.describe()) # summary statistics for numerical columns

          Magnitude    Depth (km)
count  14726.000000  14726.000000
mean       4.465259     94.491783
std        1.048695    116.171875
min        0.700000      0.000000
25%        3.700000     11.000000
50%        4.600000     60.000000
75%        5.200000    134.000000
max       37.000000   3654.000000


In [ ]:
print(df.isnull().sum()) # number of missing values in each column

Date            0
Time (utc)      0
Region          0
Magnitude       0
Depth (km)      0
Latitude        0
Longitude       0
Mode            0
Map           128
dtype: int64
